<a href="https://colab.research.google.com/github/noushsuon/honours-project/blob/main/RoBERTa-Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install comet-ml
!pip install datasets
!pip install evaluate
!pip install accelerate -U
!pip install codecarbon
!pip install transformers
!pip install tokenizers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
experiment.end()

In [ ]:
import comet_ml
from comet_ml import Experiment
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

experiment = Experiment(api_key="uyaIbaxDfy7jK9j7u4KNxeKNg",project_name="Honours",)
comet_ml.init(api_key='uyaIbaxDfy7jK9j7u4KNxeKNg',project_name='comet-bert-experiment')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
[codecarbon INFO @ 19:12:34] [setup] RAM Tracking...
[codecarbon INFO @ 19:12:34] [setup] GPU Tracking...
[codecarbon INFO @ 19:12:34] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 19:12:34] [setup] CPU Tracking...
[codecarbon WARNING @ 19:12:34] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 19:12:35] We saw that you have a 13th Gen Intel(R) Core(TM) i9-13950HX but we don't know it. Please contact us.
[codecarbon INFO @ 19:12:35] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i9-13950HX
[codecarbon INFO @ 19:12:35] >>> Tracker's metadata:
[codecarbon INFO @ 19:12:35]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 19:12:35]   Python version: 3.10.12
[codecarbon INFO @ 19:12:35]   CodeCarbon version: 2.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

raw_datasets = load_dataset("squad")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [ ]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s>To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?</s></s>Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.</s>'

In [ ]:
max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second", #only truncates the context
        stride=stride,
        return_overflowing_tokens=True, #model will store strings ignored when truncating
        return_offsets_mapping=True, #stores the location of tokens in original text
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
#use previously defined function to preprocess training data
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset) #check the sentences have been tokenised

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

[codecarbon INFO @ 19:12:50] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:12:50] Energy consumed for all GPUs : 0.000027 kWh. Total GPU Power : 6.571824638899391 W
[codecarbon INFO @ 19:12:50] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:12:50] 0.000229 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:13:05] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:13:05] Energy consumed for all GPUs : 0.000057 kWh. Total GPU Power : 7.006012940652041 W
[codecarbon INFO @ 19:13:05] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:13:05] 0.000459 kWh of electricity used since the beginning.


(87599, 88567)

In [ ]:
'''
similar to training data except
  -without start and end positions
  -simplified offset mapping
  -uses example ids
'''
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
#use function to preprocess validation data
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

[codecarbon INFO @ 19:13:20] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:13:20] Energy consumed for all GPUs : 0.000098 kWh. Total GPU Power : 9.873183289313626 W


(10570, 10790)

[codecarbon INFO @ 19:13:20] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:13:20] 0.000701 kWh of electricity used since the beginning.


post processing

In [ ]:
import evaluate
metric = evaluate.load("squad")

In [ ]:
from tqdm.auto import tqdm
import collections
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

def get_example(index):
    return tokenized_datasets["test"][index]["text"]

def compute_metrics(start_logits, end_logits, features, examples):
    experiment = comet_ml.get_global_experiment()
    example_to_features = collections.defaultdict(list)

    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []

    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples
    ]
    metric_result = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    if experiment:
        #experiment.set_epoch(epoch)
        experiment.log_metric("f1", metric_result['f1'])
        experiment.log_metric("exact_match", metric_result['exact_match'])
    return metric_result

In [ ]:
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

[codecarbon INFO @ 19:13:35] Energy consumed for RAM : 0.000096 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:13:35] Energy consumed for all GPUs : 0.000170 kWh. Total GPU Power : 17.352534889607647 W
[codecarbon INFO @ 19:13:35] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:13:35] 0.000975 kWh of electricity used since the beginning.
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import transformers
from transformers import TrainingArguments
import accelerate

%env COMET_MODE=ONLINE
%env COMET_LOG_ASSETS=TRUE

args = TrainingArguments(
    "roberta-finetuned-squad",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=100,
    fp16=True,
    push_to_hub=True,
    report_to=["comet_ml"],
    logging_strategy="epoch",
    seed=42,
    load_best_model_at_end=False,
    save_total_limit=1,
)

env: COMET_MODE=ONLINE
env: COMET_LOG_ASSETS=TRUE


In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
[codecarbon INFO @ 19:13:47] Energy consumed for RAM : 0.000115 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:13:47] Energy consumed for all GPUs : 0.000210 kWh. Total GPU Power : 12.478892076762676 W
[codecarbon INFO @ 19:13:47] Energy consumed for all CPUs : 0.000846 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:13:47] 0.001171 kWh of electricity used since the beginning.
COMET INFO: ---------------------------------------------------------------------------------------
CO

Epoch,Training Loss,Validation Loss
1,1.058500,No log
2,0.696700,No log
3,0.517800,No log


[codecarbon INFO @ 19:14:06] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:14:06] Energy consumed for all GPUs : 0.000442 kWh. Total GPU Power : 106.15049674552944 W
[codecarbon INFO @ 19:14:06] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:14:06] 0.000644 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:14:21] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:14:21] Energy consumed for all GPUs : 0.000923 kWh. Total GPU Power : 115.34663149156181 W
[codecarbon INFO @ 19:14:21] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 19:14:21] 0.001326 kWh of electricity used since the beginning.
[codecarbon INFO @ 19:14:36] Energy consumed for RAM : 0.000072 kWh. RAM Power : 5.789747714996338 W
[codecarbon INFO @ 19:14:36] Energy consumed for all GPUs : 0.001404 kWh. Total GPU Power : 115.52787303447

TrainOutput(global_step=33213, training_loss=0.7576241001499789, metrics={'train_runtime': 3466.4215, 'train_samples_per_second': 76.65, 'train_steps_per_second': 9.581, 'total_flos': 5.207010717113395e+16, 'train_loss': 0.7576241001499789, 'epoch': 3.0})

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/10570 [00:00<?, ?it/s]

{'exact_match': 85.88457899716178, 'f1': 92.23319187313736}

In [ ]:
trainer.push_to_hub(commit_message="Training complete")

'https://huggingface.co/noushsuon/bert-finetuned-squad/tree/main/'

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name
model_name = "roberta-finetuned-squad"
repo_name = get_full_repo_name(model_name)
repo_name

'noushsuon/bert-finetuned-squad'

In [ ]:
from huggingface_hub import Repository
output_dir = "roberta-finetuned-squad-accelerate" + model_checkpoint

repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/noushsuon/bert-finetuned-squad into local empty directory.


Download file model.safetensors:   0%|          | 18.4k/473M [00:00<?, ?B/s]

Download file runs/Jan31_16-09-00_dcaff34ff298/events.out.tfevents.1706874684.dcaff34ff298.1318.0: 100%|######…

Download file training_args.bin: 100%|##########| 4.80k/4.80k [00:00<?, ?B/s]

Clean file runs/Jan31_16-09-00_dcaff34ff298/events.out.tfevents.1706874684.dcaff34ff298.1318.0:   7%|6        …

...

Clean file runs/Feb19_21-04-07_a1e3143d30be/events.out.tfevents.1708376647.a1e3143d30be.826.0:  13%|#3        …

Clean file runs/Mar01_10-58-51_9c4a085fe430/events.out.tfevents.1709290732.9c4a085fe430.438.0:  22%|##2       …

Download file runs/Mar01_17-52-21_9c4a085fe430/events.out.tfevents.1709315550.9c4a085fe430.438.1: 100%|#######…

Clean file runs/Mar01_17-52-21_9c4a085fe430/events.out.tfevents.1709315550.9c4a085fe430.438.1:  24%|##4       …

Clean file model.safetensors:   0%|          | 1.00k/473M [00:00<?, ?B/s]

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )

    for key, value in metrics.items():
        experiment.log_metric(key, value)

    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/33213 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/1349 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 0: {'exact_match': 84.62630085146641, 'f1': 91.40146497396202}
Evaluation!


...

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 1: {'exact_match': 85.6480605487228, 'f1': 91.93764161717085}
Evaluation!


  0%|          | 0/1349 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

epoch 2: {'exact_match': 86.39545884578997, 'f1': 92.48997444570277}


In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

In [ ]:
from transformers import pipeline

model_checkpoint = "noushsuon/roberta-finetuned-squad"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """
Bill is Andrew's father
"""
question = "Who is Bill?"
answer = question_answerer(question=question, context=context)

experiment.log_text("Question", question)
experiment.log_text("Context", context)
experiment.log_text("Answer", answer)

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
context = """
Bill is Andrew's father
"""
question = "Who is Bill's father?"
answer = question_answerer(question=question, context=context)
print(context, question)
print(answer)


Bill is Andrew's father
 Who is Bill's father?
{'score': 0.9394617080688477, 'start': 9, 'end': 15, 'answer': 'Andrew'}


In [ ]:
context = """
Bill is Andrew's father
"""
question = "Who is Andrew's son?"
answer = question_answerer(question=question, context=context)
print(context, question)
print(answer)


Bill is Andrew's father
 Who is Andrew's son?
{'score': 0.9577686190605164, 'start': 1, 'end': 5, 'answer': 'Bill'}
